<a href="https://colab.research.google.com/github/BaseKan/aiday_training_resources/blob/main/Cython/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Tutorial

*Deze tutorial is een aangepaste versie van de officiële [Cython tutorial](https://cython.readthedocs.io/en/latest/src/tutorial/cython_tutorial.html) en de [Cython in Jupyter Notebooks instructies](https://cython.readthedocs.io/en/latest/src/quickstart/build.html#using-the-jupyter-notebook). Het compileren van Cython files voor gebruik in reguliere python projecten is in dit notebook achterwege gelaten. Dit zal tijdens de kennissessie aan bod komen.*

## The Basics of Cython

The fundamental nature of Cython can be summed up as follows: Cython is Python with C data types.

Cython is Python: Almost any piece of Python code is also valid Cython code. (There are a few Limitations, but this approximation will serve for now.) The Cython compiler will convert it into C code which makes equivalent calls to the Python/C API.

But Cython is much more than that, because parameters and variables can be declared to have C data types. Code which manipulates Python values and C values can be freely intermixed, with conversions occurring automatically wherever possible. Reference count maintenance and error checking of Python operations is also automatic, and the full power of Python’s exception handling facilities, including the try-except and try-finally statements, is available to you – even in the midst of manipulating C data.

## Cython and Jupyter Notebooks

We will use the following python code for our first example. 

In [7]:
a = 0
for i in range(10):
    a += i
print(a)

45


Cython can be used conveniently and interactively from a web browser through the Jupyter notebook. To enable support for Cython compilation load the Cython extension from within the Jupyter notebook:

In [ ]:
%load_ext Cython

Then, prefix a cell with the %%cython marker to compile it with Cython:

In [8]:
%%cython

cdef int a = 0
for i in range(10):
    a += i
print(a)

45


## Fibonacci Fun


From the official Python tutorial a simple fibonacci function is defined as:

In [2]:
def fib(n):
    """Print the Fibonacci series up to n."""
    a, b = 0, 1
    while b < n:
        print(b, end=' ')
        a, b = b, a + b

    print()

In [3]:
fib(2000)

1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 
